In [29]:
from collections import Counter
import numpy as np
from threading import Thread
from multiprocessing import Process, Manager, freeze_support, set_start_method, Array, Queue
from time import time, sleep
import concurrent.futures
from find_summ_helper import find_sum
import multiprocessing, math
import pandas as pd

In [30]:
def find_average_sequential(A):
    return sum(A)/len(A)

In [31]:
def find_sum_for_slice(A, start, end, res):
    s = 0
    for i in range(start,end):
        s+=A[i]
    res.append(s)


def find_average_threads(A,cores):
    N = len(A)
    steps = math.ceil(N//cores)#number of rows should be processed by each thread
    start = 0
    threads = []
    res = []

    if __name__ == '__main__':
        while start<N:
            end = N if start+steps > N else start+steps
            t = Thread(target=find_sum_for_slice, args = [A,start,end,res])
            threads.append(t)
            start = end #update row number for the start of next process
        for t in threads: t.start()
        for t in threads: t.join()
        return sum(res)/N

In [32]:
def find_average_multiprocessing(A,cores):
    N = len(A)
    #res = []
    steps = N//cores
    start = 0
    processes = []
    
    if __name__ == '__main__':
        queue = Queue()
        for c in range(cores):
            end = start+steps
            if cores-c==1:end = N
            p = Process(target=find_sum, args = [A[start:end],queue])
            processes.append(p)
            start = end
        for p in processes: p.start()
        for p in processes: p.join()
        curr_sum = 0
        for c in range(cores):curr_sum+=queue.get()
        #print(return_dict.values())
        return curr_sum/N

In [39]:
def compare_proc_sizes_for_N(N):
    procs = [1,2,3,4]
    df_compare_add_value = pd.DataFrame(columns=['Process count','Sequential','Multithreading','Multiprocessing'])
    A = np.random.random(size = (N))
    for i in range(len(procs)):
        proc = procs[i]
        t0 = time()
        C = find_average_sequential(A)
        t1 = time()
        #print(C)
    
        D = find_average_threads(A,proc)
        t2 = time()
        #print(D)

        E = find_average_multiprocessing(A,proc)
        t3 = time()
        #print(E)

        df_compare_add_value.loc[i] = [proc, t1-t0, t2-t1, t3-t2]
    return df_compare_add_value


In [40]:
df_pow_4 = compare_proc_sizes_for_N(pow(10,4))
df_pow_4

,Process count,Sequential,Multithreading,Multiprocessing
0,1.0,0.002589,0.004443,0.264874
1,2.0,0.001829,0.003581,0.293277
2,3.0,0.001891,0.004008,0.377122
3,4.0,0.002029,0.004071,0.477128


In [65]:
df_pow_5 = compare_proc_sizes_for_N(pow(10,5))
df_pow_5

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects
0,1.0,0.053282,0.084128,0.295096
1,2.0,0.040613,0.084628,0.515870
2,3.0,0.043122,0.084012,0.764834
3,4.0,0.040070,0.083333,0.990363


In [41]:
df_pow_6 = compare_proc_sizes_for_N(pow(10,6))
df_pow_6

,Process count,Sequential,Multithreading,Multiprocessing
0,1.0,0.213291,0.322138,0.463816
1,2.0,0.219649,0.311623,0.610807
2,3.0,0.208589,0.335388,0.787273
3,4.0,0.185905,0.300346,1.002053


In [42]:
df_pow_7 = compare_proc_sizes_for_N(pow(10,7))
df_pow_7

,Process count,Sequential,Multithreading,Multiprocessing
0,1.0,1.959792,3.549002,2.628768
1,2.0,2.087877,3.224754,1.772896
2,3.0,1.979509,3.155005,1.911474
3,4.0,2.037094,3.384455,2.580634


In [43]:
df_pow_8 = compare_proc_sizes_for_N(pow(10,8))
df_pow_8

,Process count,Sequential,Multithreading,Multiprocessing
0,1.0,20.890021,32.149124,24.084989
1,2.0,21.997558,31.806075,14.287500
2,3.0,20.284546,32.940196,13.691156
3,4.0,19.261941,32.390067,15.400621
